<a href="https://colab.research.google.com/github/careapp7/recommend-page/blob/main/%E8%BF%94%E4%BF%A1%E9%80%9F%E5%BA%A6%E8%A8%88%E7%AE%97_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title
# 返信分析ダッシュボード v3.3 (UI・UX改善版) - 共有ドライブ「メールログ」直下対応 / Dropdownはファイル名表示 / ステータス超目立つ
!pip -q install gradio japanize-matplotlib

import os, glob, base64, io
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import japanize_matplotlib
import gradio as gr

# =========================
# 1. 環境設定・Drive連携（共有ドライブ対応）
# =========================
LOCAL_CACHE_DIR = "/content/.mail_log_cache"
os.makedirs(LOCAL_CACHE_DIR, exist_ok=True)

SHARED_DRIVE_NAME = "メールログ"
SHARED_DRIVE_BASE = f"/content/drive/Shareddrives/{SHARED_DRIVE_NAME}"

def ensure_drive_mounted():
    if os.path.exists("/content/drive"):
        return None
    try:
        from google.colab import drive
        drive.mount("/content/drive", force_remount=True)
        return None
    except:
        return "❌ Google Drive をマウントできませんでした。"

def get_csv_candidates():
    ensure_drive_mounted()
    base = SHARED_DRIVE_BASE

    cands = []
    if os.path.exists(base):
        cands = glob.glob(os.path.join(base, "**", "*.csv"), recursive=True)

    cands.sort(key=os.path.getmtime, reverse=True)
    return cands

# =========================
# 2. 分析・集計ロジック（最初メッセージ基準）
# =========================
def analyze_data_all(df, pid):
    df["パフォーマ"] = pd.to_numeric(df["パフォーマ"], errors="coerce")
    df_p = df[df["パフォーマ"] == pid].copy()
    if df_p.empty:
        return None, f"❌ ID {pid} のデータが見つかりません。"

    df_p["送信日時"] = pd.to_datetime(df_p["送信日時"], errors="coerce")
    df_p = df_p.dropna(subset=["送信日時"]).sort_values("送信日時")
    df_p["hour"] = df_p["送信日時"].dt.hour

    member_only = df_p[df_p["送信方向"] == "会員→パフォーマ"]
    vip_top = (
        member_only.groupby("会員")
        .size()
        .reset_index(name="送信数")
        .sort_values("送信数", ascending=False)
        .head(10)
    )

    activity_stats = df_p.groupby(["hour", "送信方向"]).size().unstack(fill_value=0)
    for col in ["会員→パフォーマ", "パフォーマ→会員"]:
        if col not in activity_stats.columns:
            activity_stats[col] = 0

    df_p = df_p.sort_values(["会員", "送信日時"])
    df_p["new_block"] = (df_p["会員"] != df_p["会員"].shift(1)) | (df_p["送信方向"] != df_p["送信方向"].shift(1))
    df_p["block_id"] = df_p["new_block"].cumsum()

    blocks = df_p.groupby("block_id").agg({"会員": "first", "送信方向": "first", "送信日時": "min"}).reset_index()
    blocks["prev_dir"] = blocks["送信方向"].shift(1)
    blocks["prev_time"] = blocks["送信日時"].shift(1)
    blocks["prev_mem"] = blocks["会員"].shift(1)

    replies = blocks[
        (blocks["送信方向"] == "パフォーマ→会員")
        & (blocks["prev_dir"] == "会員→パフォーマ")
        & (blocks["会員"] == blocks["prev_mem"])
    ].copy()

    if len(replies) == 0:
        return None, "❌ 返信ペアが不足しています。"

    replies["wait_min"] = (replies["送信日時"] - replies["prev_time"]).dt.total_seconds() / 60
    replies = replies[replies["wait_min"] < 1440]

    bins = [0, 5, 10, 15, 30, 999999]
    labels = ["5分以内", "5-10分", "10-15分", "15-30分", "30分超"]
    replies["category"] = pd.cut(replies["wait_min"], bins=bins, labels=labels, right=True)
    category_counts = replies["category"].value_counts().reindex(labels, fill_value=0)

    hourly_speed = replies.groupby(replies["送信日時"].dt.hour)["wait_min"].agg(["median", "mean", "count"]).reset_index()
    hourly_speed.columns = ["hour", "median", "mean", "count"]
    hourly_speed = pd.merge(pd.DataFrame({"hour": range(24)}), hourly_speed, on="hour", how="left").fillna(0)

    res = {
        "vip_top": vip_top,
        "activity_stats": activity_stats,
        "replies": replies,
        "category_counts": category_counts,
        "hourly_speed": hourly_speed,
        "total_count": len(replies),
        "median_min": replies["wait_min"].median(),
        "mean_min": replies["wait_min"].mean(),
    }
    return res, None

# =========================
# 3. 描画
# =========================
def fig_to_html(fig):
    buf = io.BytesIO()
    fig.savefig(buf, format="png", bbox_inches="tight")
    buf.seek(0)
    data = base64.b64encode(buf.read()).decode("utf-8")
    plt.close(fig)
    return f'<div style="text-align:center;"><img src="data:image/png;base64,{data}" style="max-width:100%; height:auto;"></div>'

def create_initial_report(pid, res):
    plt.style.use("default")
    japanize_matplotlib.japanize()

    color_bg = "#F7F9FC"
    color_card = "#FFFFFF"
    color_primary = "#4e73df"
    color_success = "#1cc88a"
    color_info = "#36b9cc"
    color_danger = "#e74a3b"
    color_text = "#2c3e50"

    fig = plt.figure(figsize=(18, 11), dpi=150, facecolor=color_bg)
    gs = fig.add_gridspec(3, 2, height_ratios=[0.1, 0.4, 0.5])

    ax_h = fig.add_subplot(gs[0, :]); ax_h.axis("off")
    ax_h.text(0.5, 0.5, f"返信速度 分析レポート (最初メッセージ基準): ID {pid}",
              ha="center", va="center", fontsize=24, fontweight="bold", color=color_text)

    def draw_card(ax, x, y, w, h, title, value, subtext, bar_color):
        ax.add_patch(patches.FancyBboxPatch((x, y), w, h, boxstyle="round,pad=0.02", ec="none", fc=color_card))
        ax.add_patch(patches.FancyBboxPatch((x, y), 0.015, h, boxstyle="round,pad=0.02", ec="none", fc=bar_color))
        ax.text(x+0.05, y+0.7*h, title, fontsize=12, color=color_primary, fontweight="bold")
        ax.text(x+0.05, y+0.35*h, value, fontsize=22, fontweight="bold")
        ax.text(x+0.05, y+0.1*h, subtext, fontsize=10, color="#858796")

    def fmt(m):
        return f"{int(m)}分{int((m-int(m))*60)}秒" if m < 60 else f"{int(m//60)}時間{int(m%60)}分"

    ax_k = fig.add_subplot(gs[1, 0]); ax_k.axis("off")
    draw_card(ax_k, 0.05, 0.55, 0.42, 0.35, "総返信数", f"{res['total_count']:,} 件", "分析対象のペア数", color_primary)
    draw_card(ax_k, 0.53, 0.55, 0.42, 0.35, "返信速度 (中央値)", fmt(res['median_min']), f"平均: {fmt(res['mean_min'])}", color_success)
    draw_card(ax_k, 0.05, 0.1, 0.42, 0.35, "爆速返信率", f"{res['category_counts']['5分以内']/res['total_count']*100:.1f}%", "5分以内の割合", color_info)
    draw_card(ax_k, 0.53, 0.1, 0.42, 0.35, "放置率 (30分超)", f"{res['category_counts']['30分超']/res['total_count']*100:.1f}%", "30分以上待たせた割合", color_danger)

    ax_p = fig.add_subplot(gs[1, 1])
    ax_p.pie(
        res['category_counts'].values,
        labels=res['category_counts'].index,
        autopct="%1.1f%%",
        startangle=90,
        counterclock=False,
        colors=[color_info, color_primary, "#858796", "#f6c23e", color_danger],
        wedgeprops={"edgecolor":"white","linewidth":3}
    )
    ax_p.add_artist(plt.Circle((0,0), 0.65, fc=color_bg))

    ax_b = fig.add_subplot(gs[2, :])
    spd = res['hourly_speed']
    b_colors = [color_danger if v > 30 else color_primary for v in spd["median"]]
    ax_b.bar(spd["hour"], spd["median"], color=b_colors, alpha=0.9, label="中央値")
    ax_b.plot(spd["hour"], spd["mean"].where(spd["count"] > 0, np.nan), marker="o", color=color_info, linewidth=2.5, label="平均")
    ax_b.set_title("時間帯別の返信スピード傾向（棒：中央値 / 線：平均）(赤色は30分以上)", fontsize=14, fontweight="bold", loc="left")
    ax_b.set_xticks(range(24)); ax_b.grid(axis="y", linestyle="--", alpha=0.5)
    return fig_to_html(fig)

def create_activity_report(pid, res):
    plt.style.use("default")
    japanize_matplotlib.japanize()

    fig = plt.figure(figsize=(15, 12), dpi=120, facecolor="#F8F9FA")
    gs = fig.add_gridspec(2, 1, hspace=0.3)

    ax_a = fig.add_subplot(gs[0, 0])
    act = res['activity_stats'].reindex(range(24), fill_value=0)
    ax_a.fill_between(act.index, act["会員→パフォーマ"], alpha=0.3, label="会員の活動（受信）")
    ax_a.plot(act.index, act["パフォーマ→会員"], linewidth=2, linestyle="--", label="あなたの送信")
    ax_a.set_title("24時間のメッセージリズム"); ax_a.set_xticks(range(24)); ax_a.legend()

    ax_v = fig.add_subplot(gs[1, 0])
    v = res['vip_top'].sort_values("送信数", ascending=True)
    ax_v.barh(v["会員"].astype(str), v["送信数"])
    ax_v.set_title("会員からの送信数 TOP10"); ax_v.grid(axis='x', linestyle='--', alpha=0.5)
    return fig_to_html(fig)

# =========================
# 4. ステータスHTML（目立つ）
# =========================
def status_html(kind: str, text: str) -> str:
    # kind: ready / running / done / error
    palette = {
        "ready":  ("#0f5132", "#d1e7dd", "#badbcc", "✅"),
        "running":("#664d03", "#fff3cd", "#ffecb5", "⏳"),
        "done":   ("#084298", "#cfe2ff", "#b6d4fe", "🎉"),
        "error":  ("#842029", "#f8d7da", "#f5c2c7", "⚠️"),
    }
    fg, bg, bd, icon = palette.get(kind, ("#333", "#eee", "#ddd", "ℹ️"))
    return f"""
    <div style="
        padding:14px 16px;
        border-radius:14px;
        border:2px solid {bd};
        background:{bg};
        box-shadow:0 6px 18px rgba(0,0,0,0.08);
        display:flex;
        align-items:center;
        gap:12px;
        ">
        <div style="font-size:28px; line-height:1;">{icon}</div>
        <div style="color:{fg};">
            <div style="font-size:20px; font-weight:800; letter-spacing:0.5px;">{text}</div>
            <div style="font-size:12px; opacity:0.9; margin-top:2px;">CSVとパフォーマIDを確認して実行できます</div>
        </div>
    </div>
    """

# =========================
# 5. アプリ実行
# =========================
def run_full_analysis(csv_path, up_file, pid_val, progress=gr.Progress()):
    try:
        if not pid_val or str(pid_val).strip() == "":
            return status_html("error", "パフォーマIDを入力してください"), None, None, None
        pid = int(pid_val)

        path = up_file.name if up_file else csv_path
        if not path or not os.path.exists(path):
            return status_html("error", f"CSVが見つかりません: {os.path.basename(str(path))}"), None, None, None

        progress(0.2, desc="データ読み込み中...")
        df = pd.read_csv(path, usecols=["パフォーマ", "会員", "送信方向", "送信日時"], low_memory=False)

        progress(0.5, desc="全指標を解析中...")
        res, err = analyze_data_all(df, pid)
        if err:
            return status_html("error", err.replace("❌", "").strip()), None, None, None

        progress(0.7, desc="返信速度レポート作成...")
        h_speed = create_initial_report(pid, res)

        progress(0.9, desc="活動傾向レポート作成...")
        h_activity = create_activity_report(pid, res)

        base_url = "https://own-mng.careapp.jp/flax/manage/ShowPerformerMailLog.do"
        vip_df = res['vip_top'].copy()
        vip_df["会員ID"] = vip_df["会員"].astype(str)
        vip_df["ログ確認"] = vip_df["会員ID"].apply(lambda m_id: f"[ログを開く]({base_url}?performerCode={pid}&memberCode={m_id})")

        return status_html("done", "分析完了。結果タブを確認してください"), h_speed, h_activity, vip_df[["会員ID", "送信数", "ログ確認"]]
    except Exception as e:
        return status_html("error", f"エラー: {str(e)}"), None, None, None

# =========================
# 6. UI構築（Dropdownはファイル名表示 / ステータス超目立つ）
# =========================
cands = get_csv_candidates()
choices = [(os.path.basename(p), p) for p in cands]  # 表示名, 値(絶対パス)

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 📊 統合ログ分析ダッシュボード v3.3（共有ドライブ：メールログ）")

    with gr.Tabs():
        with gr.TabItem("1. 設定と実行"):
            with gr.Row():
                with gr.Column():
                    file_dropdown = gr.Dropdown(
                        choices=choices,
                        value=choices[0][1] if choices else None,
                        label="CSV選択（ファイル名表示）"
                    )
                    file_upload = gr.File(label="直接アップロード（代替）", file_types=[".csv"])
                with gr.Column():
                    pid_input = gr.Textbox(label="パフォーマIDを入力", placeholder="例: 12345")
                    run_btn = gr.Button("🚀 分析を開始する", variant="primary", size="lg")
                    status_output = gr.HTML(status_html("ready", "準備OK"))

        with gr.TabItem("2. 返信速度レポート"):
            html_speed_out = gr.HTML()

        with gr.TabItem("3. 活動傾向 ＆ VIP"):
            with gr.Row():
                with gr.Column(scale=2):
                    html_activity_out = gr.HTML()
                with gr.Column(scale=1):
                    table_vip_out = gr.Dataframe(
                        headers=["会員ID", "送信数", "ログ確認"],
                        datatype=["str", "number", "markdown"],
                        interactive=False
                    )

    run_btn.click(
        fn=lambda: status_html("running", "分析中..."),
        outputs=status_output
    ).then(
        fn=run_full_analysis,
        inputs=[file_dropdown, file_upload, pid_input],
        outputs=[status_output, html_speed_out, html_activity_out, table_vip_out]
    )

# =========================
# 7. 起動
# =========================
port = 7860
while True:
    try:
        demo.launch(
            share=False,
            inline=False,
            server_name="0.0.0.0",
            server_port=port,
            prevent_thread_lock=True
        )
        break
    except:
        port += 1

from google.colab import output
output.serve_kernel_port_as_iframe(port, width="100%", height=1000)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 20.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Mounted at /content/drive


/tmp/ipython-input-3468626640.py:273: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft()) as demo:


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [ ]:
# @title
# 返信分析ダッシュボード v3.3 (UI・UX改善版) - 共有ドライブ「メールログ」直下対応 / Dropdownはファイル名表示 / ステータス超目立つ
!pip -q install gradio japanize-matplotlib

import os, glob, base64, io
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import japanize_matplotlib
import gradio as gr

# =========================
# 1. 環境設定・Drive連携（共有ドライブ対応）
# =========================
LOCAL_CACHE_DIR = "/content/.mail_log_cache"
os.makedirs(LOCAL_CACHE_DIR, exist_ok=True)

SHARED_DRIVE_NAME = "メールログ"
SHARED_DRIVE_BASE = f"/content/drive/Shareddrives/{SHARED_DRIVE_NAME}"

def ensure_drive_mounted():
    if os.path.exists("/content/drive"):
        return None
    try:
        from google.colab import drive
        drive.mount("/content/drive", force_remount=True)
        return None
    except:
        return "❌ Google Drive をマウントできませんでした。"

def get_csv_candidates():
    ensure_drive_mounted()
    base = SHARED_DRIVE_BASE

    cands = []
    if os.path.exists(base):
        cands = glob.glob(os.path.join(base, "**", "*.csv"), recursive=True)

    cands.sort(key=os.path.getmtime, reverse=True)
    return cands

# =========================
# 2. 分析・集計ロジック（最初メッセージ基準）
# =========================
def analyze_data_all(df, pid):
    df["パフォーマ"] = pd.to_numeric(df["パフォーマ"], errors="coerce")
    df_p = df[df["パフォーマ"] == pid].copy()
    if df_p.empty:
        return None, f"❌ ID {pid} のデータが見つかりません。"

    df_p["送信日時"] = pd.to_datetime(df_p["送信日時"], errors="coerce")
    df_p = df_p.dropna(subset=["送信日時"]).sort_values("送信日時")
    df_p["hour"] = df_p["送信日時"].dt.hour

    member_only = df_p[df_p["送信方向"] == "会員→パフォーマ"]
    vip_top = (
        member_only.groupby("会員")
        .size()
        .reset_index(name="送信数")
        .sort_values("送信数", ascending=False)
        .head(10)
    )

    activity_stats = df_p.groupby(["hour", "送信方向"]).size().unstack(fill_value=0)
    for col in ["会員→パフォーマ", "パフォーマ→会員"]:
        if col not in activity_stats.columns:
            activity_stats[col] = 0

    df_p = df_p.sort_values(["会員", "送信日時"])
    df_p["new_block"] = (df_p["会員"] != df_p["会員"].shift(1)) | (df_p["送信方向"] != df_p["送信方向"].shift(1))
    df_p["block_id"] = df_p["new_block"].cumsum()

    blocks = df_p.groupby("block_id").agg({"会員": "first", "送信方向": "first", "送信日時": "min"}).reset_index()
    blocks["prev_dir"] = blocks["送信方向"].shift(1)
    blocks["prev_time"] = blocks["送信日時"].shift(1)
    blocks["prev_mem"] = blocks["会員"].shift(1)

    replies = blocks[
        (blocks["送信方向"] == "パフォーマ→会員")
        & (blocks["prev_dir"] == "会員→パフォーマ")
        & (blocks["会員"] == blocks["prev_mem"])
    ].copy()

    if len(replies) == 0:
        return None, "❌ 返信ペアが不足しています。"

    replies["wait_min"] = (replies["送信日時"] - replies["prev_time"]).dt.total_seconds() / 60
    replies = replies[replies["wait_min"] < 1440]

    bins = [0, 5, 10, 15, 30, 999999]
    labels = ["5分以内", "5-10分", "10-15分", "15-30分", "30分超"]
    replies["category"] = pd.cut(replies["wait_min"], bins=bins, labels=labels, right=True)
    category_counts = replies["category"].value_counts().reindex(labels, fill_value=0)

    hourly_speed = replies.groupby(replies["送信日時"].dt.hour)["wait_min"].agg(["median", "mean", "count"]).reset_index()
    hourly_speed.columns = ["hour", "median", "mean", "count"]
    hourly_speed = pd.merge(pd.DataFrame({"hour": range(24)}), hourly_speed, on="hour", how="left").fillna(0)

    res = {
        "vip_top": vip_top,
        "activity_stats": activity_stats,
        "replies": replies,
        "category_counts": category_counts,
        "hourly_speed": hourly_speed,
        "total_count": len(replies),
        "median_min": replies["wait_min"].median(),
        "mean_min": replies["wait_min"].mean(),
    }
    return res, None

# =========================
# 3. 描画
# =========================
def fig_to_html(fig):
    buf = io.BytesIO()
    fig.savefig(buf, format="png", bbox_inches="tight")
    buf.seek(0)
    data = base64.b64encode(buf.read()).decode("utf-8")
    plt.close(fig)
    return f'<div style="text-align:center;"><img src="data:image/png;base64,{data}" style="max-width:100%; height:auto;"></div>'

def create_initial_report(pid, res):
    plt.style.use("default")
    japanize_matplotlib.japanize()

    color_bg = "#F7F9FC"
    color_card = "#FFFFFF"
    color_primary = "#4e73df"
    color_success = "#1cc88a"
    color_info = "#36b9cc"
    color_danger = "#e74a3b"
    color_text = "#2c3e50"

    fig = plt.figure(figsize=(18, 11), dpi=150, facecolor=color_bg)
    gs = fig.add_gridspec(3, 2, height_ratios=[0.1, 0.4, 0.5])

    ax_h = fig.add_subplot(gs[0, :]); ax_h.axis("off")
    ax_h.text(0.5, 0.5, f"返信速度 分析レポート (最初メッセージ基準): ID {pid}",
              ha="center", va="center", fontsize=24, fontweight="bold", color=color_text)

    def draw_card(ax, x, y, w, h, title, value, subtext, bar_color):
        ax.add_patch(patches.FancyBboxPatch((x, y), w, h, boxstyle="round,pad=0.02", ec="none", fc=color_card))
        ax.add_patch(patches.FancyBboxPatch((x, y), 0.015, h, boxstyle="round,pad=0.02", ec="none", fc=bar_color))
        ax.text(x+0.05, y+0.7*h, title, fontsize=12, color=color_primary, fontweight="bold")
        ax.text(x+0.05, y+0.35*h, value, fontsize=22, fontweight="bold")
        ax.text(x+0.05, y+0.1*h, subtext, fontsize=10, color="#858796")

    def fmt(m):
        return f"{int(m)}分{int((m-int(m))*60)}秒" if m < 60 else f"{int(m//60)}時間{int(m%60)}分"

    ax_k = fig.add_subplot(gs[1, 0]); ax_k.axis("off")
    draw_card(ax_k, 0.05, 0.55, 0.42, 0.35, "総返信数", f"{res['total_count']:,} 件", "分析対象のペア数", color_primary)
    draw_card(ax_k, 0.53, 0.55, 0.42, 0.35, "返信速度 (中央値)", fmt(res['median_min']), f"平均: {fmt(res['mean_min'])}", color_success)
    draw_card(ax_k, 0.05, 0.1, 0.42, 0.35, "爆速返信率", f"{res['category_counts']['5分以内']/res['total_count']*100:.1f}%", "5分以内の割合", color_info)
    draw_card(ax_k, 0.53, 0.1, 0.42, 0.35, "放置率 (30分超)", f"{res['category_counts']['30分超']/res['total_count']*100:.1f}%", "30分以上待たせた割合", color_danger)

    ax_p = fig.add_subplot(gs[1, 1])
    ax_p.pie(
        res['category_counts'].values,
        labels=res['category_counts'].index,
        autopct="%1.1f%%",
        startangle=90,
        counterclock=False,
        colors=[color_info, color_primary, "#858796", "#f6c23e", color_danger],
        wedgeprops={"edgecolor":"white","linewidth":3}
    )
    ax_p.add_artist(plt.Circle((0,0), 0.65, fc=color_bg))

    ax_b = fig.add_subplot(gs[2, :])
    spd = res['hourly_speed']
    b_colors = [color_danger if v > 30 else color_primary for v in spd["median"]]
    ax_b.bar(spd["hour"], spd["median"], color=b_colors, alpha=0.9, label="中央値")
    ax_b.plot(spd["hour"], spd["mean"].where(spd["count"] > 0, np.nan), marker="o", color=color_info, linewidth=2.5, label="平均")
    ax_b.set_title("時間帯別の返信スピード傾向（棒：中央値 / 線：平均）(赤色は30分以上)", fontsize=14, fontweight="bold", loc="left")
    ax_b.set_xticks(range(24)); ax_b.grid(axis="y", linestyle="--", alpha=0.5)
    return fig_to_html(fig)

def create_activity_report(pid, res):
    plt.style.use("default")
    japanize_matplotlib.japanize()

    fig = plt.figure(figsize=(15, 12), dpi=120, facecolor="#F8F9FA")
    gs = fig.add_gridspec(2, 1, hspace=0.3)

    ax_a = fig.add_subplot(gs[0, 0])
    act = res['activity_stats'].reindex(range(24), fill_value=0)
    ax_a.fill_between(act.index, act["会員→パフォーマ"], alpha=0.3, label="会員の活動（受信）")
    ax_a.plot(act.index, act["パフォーマ→会員"], linewidth=2, linestyle="--", label="あなたの送信")
    ax_a.set_title("24時間のメッセージリズム"); ax_a.set_xticks(range(24)); ax_a.legend()

    ax_v = fig.add_subplot(gs[1, 0])
    v = res['vip_top'].sort_values("送信数", ascending=True)
    ax_v.barh(v["会員"].astype(str), v["送信数"])
    ax_v.set_title("会員からの送信数 TOP10"); ax_v.grid(axis='x', linestyle='--', alpha=0.5)
    return fig_to_html(fig)

# =========================
# 4. ステータスHTML（目立つ）
# =========================
def status_html(kind: str, text: str) -> str:
    # kind: ready / running / done / error
    palette = {
        "ready":  ("#0f5132", "#d1e7dd", "#badbcc", "✅"),
        "running":("#664d03", "#fff3cd", "#ffecb5", "⏳"),
        "done":   ("#084298", "#cfe2ff", "#b6d4fe", "🎉"),
        "error":  ("#842029", "#f8d7da", "#f5c2c7", "⚠️"),
    }
    fg, bg, bd, icon = palette.get(kind, ("#333", "#eee", "#ddd", "ℹ️"))
    return f"""
    <div style="
        padding:14px 16px;
        border-radius:14px;
        border:2px solid {bd};
        background:{bg};
        box-shadow:0 6px 18px rgba(0,0,0,0.08);
        display:flex;
        align-items:center;
        gap:12px;
        ">
        <div style="font-size:28px; line-height:1;">{icon}</div>
        <div style="color:{fg};">
            <div style="font-size:20px; font-weight:800; letter-spacing:0.5px;">{text}</div>
            <div style="font-size:12px; opacity:0.9; margin-top:2px;">CSVとパフォーマIDを確認して実行できます</div>
        </div>
    </div>
    """

# =========================
# 5. アプリ実行
# =========================
def run_full_analysis(csv_path, up_file, pid_val, progress=gr.Progress()):
    try:
        if not pid_val or str(pid_val).strip() == "":
            return status_html("error", "パフォーマIDを入力してください"), None, None, None
        pid = int(pid_val)

        path = up_file.name if up_file else csv_path
        if not path or not os.path.exists(path):
            return status_html("error", f"CSVが見つかりません: {os.path.basename(str(path))}"), None, None, None

        progress(0.2, desc="データ読み込み中...")
        df = pd.read_csv(path, usecols=["パフォーマ", "会員", "送信方向", "送信日時"], low_memory=False)

        progress(0.5, desc="全指標を解析中...")
        res, err = analyze_data_all(df, pid)
        if err:
            return status_html("error", err.replace("❌", "").strip()), None, None, None

        progress(0.7, desc="返信速度レポート作成...")
        h_speed = create_initial_report(pid, res)

        progress(0.9, desc="活動傾向レポート作成...")
        h_activity = create_activity_report(pid, res)

        base_url = "https://own-mng.urano.love/flax/manage/ShowPerformerMailLog.do"
        vip_df = res['vip_top'].copy()
        vip_df["会員ID"] = vip_df["会員"].astype(str)
        vip_df["ログ確認"] = vip_df["会員ID"].apply(lambda m_id: f"[ログを開く]({base_url}?performerCode={pid}&memberCode={m_id})")

        return status_html("done", "分析完了。結果タブを確認してください"), h_speed, h_activity, vip_df[["会員ID", "送信数", "ログ確認"]]
    except Exception as e:
        return status_html("error", f"エラー: {str(e)}"), None, None, None

# =========================
# 6. UI構築（Dropdownはファイル名表示 / ステータス超目立つ）
# =========================
cands = get_csv_candidates()
choices = [(os.path.basename(p), p) for p in cands]  # 表示名, 値(絶対パス)

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 📊 ウラーノ統合ログ分析ダッシュボード v3.3（共有ドライブ：メールログ）")

    with gr.Tabs():
        with gr.TabItem("1. 設定と実行"):
            with gr.Row():
                with gr.Column():
                    file_dropdown = gr.Dropdown(
                        choices=choices,
                        value=choices[0][1] if choices else None,
                        label="CSV選択（ファイル名表示）"
                    )
                    file_upload = gr.File(label="直接アップロード（代替）", file_types=[".csv"])
                with gr.Column():
                    pid_input = gr.Textbox(label="パフォーマIDを入力", placeholder="例: 12345")
                    run_btn = gr.Button("🚀 分析を開始する", variant="primary", size="lg")
                    status_output = gr.HTML(status_html("ready", "準備OK"))

        with gr.TabItem("2. 返信速度レポート"):
            html_speed_out = gr.HTML()

        with gr.TabItem("3. 活動傾向 ＆ VIP"):
            with gr.Row():
                with gr.Column(scale=2):
                    html_activity_out = gr.HTML()
                with gr.Column(scale=1):
                    table_vip_out = gr.Dataframe(
                        headers=["会員ID", "送信数", "ログ確認"],
                        datatype=["str", "number", "markdown"],
                        interactive=False
                    )

    run_btn.click(
        fn=lambda: status_html("running", "分析中..."),
        outputs=status_output
    ).then(
        fn=run_full_analysis,
        inputs=[file_dropdown, file_upload, pid_input],
        outputs=[status_output, html_speed_out, html_activity_out, table_vip_out]
    )

# =========================
# 7. 起動
# =========================
port = 7860
while True:
    try:
        demo.launch(
            share=False,
            inline=False,
            server_name="0.0.0.0",
            server_port=port,
            prevent_thread_lock=True
        )
        break
    except:
        port += 1

from google.colab import output
output.serve_kernel_port_as_iframe(port, width="100%", height=1000)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 34.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Mounted at /content/drive


/tmp/ipython-input-935724108.py:273: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft()) as demo:


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [ ]:
# @title
# 返信分析ダッシュボード v4.5（返信KPI + ポイント + 転換）
# ✅ タブ6「有料/無料送信 ＆ 有料開封」を削除
# ✅ パフォーマID未入力なら全体分析
# ✅ 会員ID型ズレ修正（全部string統一）
# ✅ 転換テーブルの列名を日本語に変更
# ✅ ポイントの「キー別合計」をドーナツ円で構成比表示（message / paymail）

!pip -q install gradio japanize-matplotlib

import os, glob, base64, io, urllib.parse as up, time
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import japanize_matplotlib
import gradio as gr

# =========================
# 1. 環境設定・Drive連携（共有ドライブ対応）
# =========================
LOCAL_CACHE_DIR = "/content/.mail_log_cache"
os.makedirs(LOCAL_CACHE_DIR, exist_ok=True)

SHARED_DRIVE_NAME = "メールログ"
SHARED_DRIVE_BASE = f"/content/drive/Shareddrives/{SHARED_DRIVE_NAME}"

def ensure_drive_mounted():
    if os.path.exists("/content/drive"):
        return None
    try:
        from google.colab import drive
        drive.mount("/content/drive", force_remount=True)
        return None
    except:
        return "❌ Google Drive をマウントできませんでした。"

def get_csv_candidates():
    ensure_drive_mounted()
    base = SHARED_DRIVE_BASE
    cands = []
    if os.path.exists(base):
        cands = glob.glob(os.path.join(base, "**", "*.csv"), recursive=True)
    cands.sort(key=os.path.getmtime, reverse=True)
    return cands

# =========================
# 2. 共通ユーティリティ
# =========================
def fig_to_html(fig):
    buf = io.BytesIO()
    fig.savefig(buf, format="png", bbox_inches="tight")
    buf.seek(0)
    data = base64.b64encode(buf.read()).decode("utf-8")
    plt.close(fig)
    return f'<div style="text-align:center;"><img src="data:image/png;base64,{data}" style="max-width:100%; height:auto;"></div>'

def status_html(kind: str, text: str, subtext: str = "CSVとパフォーマIDを確認して実行できます") -> str:
    palette = {
        "ready":  ("#0f5132", "#d1e7dd", "#badbcc", "✅"),
        "running":("#664d03", "#fff3cd", "#ffecb5", "⏳"),
        "done":   ("#084298", "#cfe2ff", "#b6d4fe", "🎉"),
        "error":  ("#842029", "#f8d7da", "#f5c2c7", "⚠️"),
    }
    fg, bg, bd, icon = palette.get(kind, ("#333", "#eee", "#ddd", "ℹ️"))
    return f"""
    <div style="
        padding:14px 16px;
        border-radius:14px;
        border:2px solid {bd};
        background:{bg};
        box-shadow:0 6px 18px rgba(0,0,0,0.08);
        display:flex;
        align-items:center;
        gap:12px;">
        <div style="font-size:28px; line-height:1;">{icon}</div>
        <div style="color:{fg};">
            <div style="font-size:20px; font-weight:800; letter-spacing:0.5px;">{text}</div>
            <div style="font-size:12px; opacity:0.9; margin-top:2px;">{subtext}</div>
        </div>
    </div>
    """

def fmt_minutes(m):
    if pd.isna(m):
        return "-"
    m = float(m)
    if m < 60:
        return f"{int(m)}分{int((m-int(m))*60)}秒"
    return f"{int(m//60)}時間{int(m%60)}分"

def _parse_qs_one(s):
    d = up.parse_qs("" if pd.isna(s) else str(s))
    return {k: (v[0] if len(v)==1 else v) for k, v in d.items()}

# =========================
# 3. 返信速度・活動傾向（最初メッセージ基準）
#    ※ pid=None（全体）のときも、会員×パフォーマでブロック化して跨ぎ事故を防ぐ
# =========================
def analyze_reply_kpi(df_mail, pid):
    need = {"パフォーマ","会員","送信方向","送信日時"}
    if not need.issubset(set(df_mail.columns)):
        return None, f"❌ メールCSVの列が不足しています。必要: {sorted(list(need))}"

    df = df_mail.copy()
    df["パフォーマ"] = pd.to_numeric(df["パフォーマ"], errors="coerce")
    df["会員"] = df["会員"].astype("string")

    df_p = df.copy() if pid is None else df[df["パフォーマ"] == pid].copy()
    if df_p.empty:
        return None, f"❌ {('全体' if pid is None else f'ID {pid}')} のメールデータが見つかりません。"

    df_p["送信日時"] = pd.to_datetime(df_p["送信日時"], errors="coerce")
    df_p = df_p.dropna(subset=["送信日時"]).sort_values("送信日時")
    df_p["hour"] = df_p["送信日時"].dt.hour

    member_only = df_p[df_p["送信方向"] == "会員→パフォーマ"]
    vip_top = (
        member_only.groupby("会員")
        .size()
        .reset_index(name="送信数")
        .sort_values("送信数", ascending=False)
        .head(10)
    )

    activity_stats = df_p.groupby(["hour", "送信方向"]).size().unstack(fill_value=0)
    for col in ["会員→パフォーマ", "パフォーマ→会員"]:
        if col not in activity_stats.columns:
            activity_stats[col] = 0

    df_p = df_p.sort_values(["パフォーマ", "会員", "送信日時"])
    df_p["new_block"] = (
        (df_p["パフォーマ"] != df_p["パフォーマ"].shift(1)) |
        (df_p["会員"] != df_p["会員"].shift(1)) |
        (df_p["送信方向"] != df_p["送信方向"].shift(1))
    )
    df_p["block_id"] = df_p["new_block"].cumsum()

    blocks = df_p.groupby("block_id").agg({
        "パフォーマ": "first",
        "会員": "first",
        "送信方向": "first",
        "送信日時": "min"
    }).reset_index()

    blocks["prev_perf"] = blocks["パフォーマ"].shift(1)
    blocks["prev_dir"]  = blocks["送信方向"].shift(1)
    blocks["prev_time"] = blocks["送信日時"].shift(1)
    blocks["prev_mem"]  = blocks["会員"].shift(1)

    replies = blocks[
        (blocks["送信方向"] == "パフォーマ→会員")
        & (blocks["prev_dir"] == "会員→パフォーマ")
        & (blocks["会員"] == blocks["prev_mem"])
        & (blocks["パフォーマ"] == blocks["prev_perf"])
    ].copy()

    if len(replies) == 0:
        return None, "❌ 返信ペアが不足しています。"

    replies["wait_min"] = (replies["送信日時"] - replies["prev_time"]).dt.total_seconds() / 60
    replies.loc[replies["wait_min"] < 0, "wait_min"] = 0

    replies_kpi = replies[replies["wait_min"] < 1440].copy()
    excluded_overday = int((replies["wait_min"] >= 1440).sum())
    if len(replies_kpi) == 0:
        return None, "❌ KPI対象（1440分未満）の返信ペアが不足しています。"

    bins = [0, 5, 10, 15, 30, 999999]
    labels = ["5分以内", "5-10分", "10-15分", "15-30分", "30分超"]
    replies_kpi["category"] = pd.cut(replies_kpi["wait_min"], bins=bins, labels=labels, right=True)
    category_counts = replies_kpi["category"].value_counts().reindex(labels, fill_value=0)

    hourly_speed = replies_kpi.groupby(replies_kpi["送信日時"].dt.hour)["wait_min"].agg(["median", "mean", "count"]).reset_index()
    hourly_speed.columns = ["hour", "median", "mean", "count"]
    hourly_speed = pd.merge(pd.DataFrame({"hour": range(24)}), hourly_speed, on="hour", how="left").fillna(0)

    replies_kpi["date"] = replies_kpi["送信日時"].dt.date
    replies_kpi["会員"] = replies_kpi["会員"].astype("string")

    reply_day_member = replies_kpi.groupby(["date","会員"])["wait_min"].agg(["median","mean","count"]).reset_index()
    reply_day_member.columns = ["date","会員ID","reply_median_min","reply_mean_min","reply_pairs"]
    reply_day_member["会員ID"] = reply_day_member["会員ID"].astype("string")

    res = {
        "vip_top": vip_top,
        "activity_stats": activity_stats,
        "replies_kpi": replies_kpi,
        "category_counts": category_counts,
        "hourly_speed": hourly_speed,
        "total_count": int(len(replies_kpi)),
        "median_min": float(replies_kpi["wait_min"].median()),
        "mean_min": float(replies_kpi["wait_min"].mean()),
        "excluded_overday": excluded_overday,
        "reply_day_member": reply_day_member,
    }
    return res, None

# =========================
# 4. ポイントログ分析（message_point_p / paymail_point_p）
# =========================
def analyze_points_split(df_points, pid):
    need = {"パフォーマー","変更日時","変更ポイント","キー","レスポンス"}
    if not need.issubset(set(df_points.columns)):
        return None, f"❌ ポイントCSVの列が不足しています。必要: {sorted(list(need))}"

    dfp = df_points.copy()
    dfp["パフォーマー"] = pd.to_numeric(dfp["パフォーマー"], errors="coerce")
    if pid is not None:
        dfp = dfp[dfp["パフォーマー"] == pid]
    if dfp.empty:
        return None, f"❌ {('全体' if pid is None else f'ID {pid}')} のポイントデータが見つかりません。"

    dfp["変更日時"] = pd.to_datetime(dfp["変更日時"], errors="coerce")
    dfp = dfp.dropna(subset=["変更日時"]).sort_values("変更日時")
    dfp["hour"] = dfp["変更日時"].dt.hour
    dfp["date"] = dfp["変更日時"].dt.date
    dfp["変更ポイント"] = pd.to_numeric(dfp["変更ポイント"], errors="coerce").fillna(0)

    parsed = dfp["レスポンス"].map(_parse_qs_one)
    dfp["会員ID"] = parsed.map(lambda d: d.get("member_code"))
    dfp["会員ID"] = dfp["会員ID"].astype("string")

    msg = dfp[dfp["キー"] == "message_point_p"].copy()
    pay = dfp[dfp["キー"] == "paymail_point_p"].copy()

    def agg_common(dfx):
        by_hour = (dfx.groupby("hour")["変更ポイント"]
                   .agg(["sum","median","count"])
                   .reindex(range(24), fill_value=0)
                   .reset_index())

        vip = (dfx.dropna(subset=["会員ID"])
               .groupby("会員ID")["変更ポイント"].sum()
               .sort_values(ascending=False).head(10)
               .reset_index())
        vip.columns = ["会員ID", "合計ポイント"]
        vip["会員ID"] = vip["会員ID"].astype("string")

        by_day_member = (dfx.dropna(subset=["会員ID"])
                         .groupby(["date","会員ID"])["変更ポイント"]
                         .agg(points_sum="sum", events_count="count")
                         .reset_index())
        by_day_member["会員ID"] = by_day_member["会員ID"].astype("string")
        return by_hour, vip, by_day_member

    msg_by_hour, msg_vip, msg_day_member = agg_common(msg)
    pay_by_hour, pay_vip, pay_day_member = agg_common(pay)

    res = {
        "raw": dfp,
        "msg": {"total_points": float(msg["変更ポイント"].sum()), "count": int(len(msg)), "by_hour": msg_by_hour, "vip": msg_vip, "day_member": msg_day_member},
        "pay": {"total_points": float(pay["変更ポイント"].sum()), "count": int(len(pay)), "by_hour": pay_by_hour, "vip": pay_vip, "day_member": pay_day_member},
    }
    return res, None

# =========================
# 5. メール分母（受信）集計
# =========================
def analyze_mail_inbound_only(df_mail, pid):
    df = df_mail.copy()
    df["パフォーマ"] = pd.to_numeric(df["パフォーマ"], errors="coerce")
    df["会員"] = df["会員"].astype("string")

    dfp = df.copy() if pid is None else df[df["パフォーマ"] == pid].copy()
    if dfp.empty:
        return None, f"❌ {('全体' if pid is None else f'ID {pid}')} のメールデータが見つかりません。"

    dfp["送信日時"] = pd.to_datetime(dfp["送信日時"], errors="coerce")
    dfp = dfp.dropna(subset=["送信日時"]).sort_values("送信日時")
    dfp["date"] = dfp["送信日時"].dt.date

    inbound = dfp[dfp["送信方向"] == "会員→パフォーマ"].copy()
    inbound_day_member = inbound.groupby(["date","会員"])["送信日時"].count().reset_index()
    inbound_day_member.columns = ["date","会員ID","inbound_count"]
    inbound_day_member["会員ID"] = inbound_day_member["会員ID"].astype("string")
    return {"inbound_day_member": inbound_day_member}, None

# =========================
# 6. 返信×ポイント転換（会員×日）
# =========================
def build_conversion(reply_day_member, inbound_day_member, points_msg_day_member, points_pay_day_member):
    base = pd.merge(inbound_day_member, reply_day_member, on=["date","会員ID"], how="left")
    base = pd.merge(base, points_msg_day_member.rename(columns={"points_sum":"msg_points","events_count":"msg_events"}), on=["date","会員ID"], how="left")
    base = pd.merge(base, points_pay_day_member.rename(columns={"points_sum":"pay_points","events_count":"pay_events"}), on=["date","会員ID"], how="left")

    for c in ["reply_median_min","reply_mean_min","reply_pairs","msg_points","msg_events","pay_points","pay_events"]:
        if c not in base.columns:
            base[c] = 0

    base[["msg_points","msg_events","pay_points","pay_events"]] = base[["msg_points","msg_events","pay_points","pay_events"]].fillna(0)
    base[["reply_median_min","reply_mean_min","reply_pairs"]] = base[["reply_median_min","reply_mean_min","reply_pairs"]].fillna(0)

    base["pay_per_msg_points"] = base.apply(lambda r: (r["pay_points"]/r["msg_points"]) if r["msg_points"]>0 else 0, axis=1)

    bins = [0, 5, 10, 15, 30, 999999]
    labels = ["5分以内", "5-10分", "10-15分", "15-30分", "30分超"]
    base["reply_bucket"] = pd.cut(base["reply_median_min"], bins=bins, labels=labels, right=True, include_lowest=True)
    base["reply_bucket"] = base["reply_bucket"].cat.add_categories(["返信ペアなし"]).fillna("返信ペアなし")
    return base

# =========================
# 7. 描画（返信/活動/ポイント/転換）
# =========================
def create_reply_report(pid, res):
    plt.style.use("default")
    japanize_matplotlib.japanize()

    color_bg = "#F7F9FC"
    color_card = "#FFFFFF"
    color_primary = "#4e73df"
    color_success = "#1cc88a"
    color_info = "#36b9cc"
    color_danger = "#e74a3b"
    color_text = "#2c3e50"

    fig = plt.figure(figsize=(18, 11), dpi=150, facecolor=color_bg)
    gs = fig.add_gridspec(3, 2, height_ratios=[0.1, 0.4, 0.5])

    ax_h = fig.add_subplot(gs[0, :]); ax_h.axis("off")
    pid_label = f"ID {pid}" if pid is not None else "全体（全パフォーマ）"
    ax_h.text(0.5, 0.5, f"返信速度 分析レポート（最初メッセージ基準）: {pid_label}",
              ha="center", va="center", fontsize=24, fontweight="bold", color=color_text)

    def draw_card(ax, x, y, w, h, title, value, subtext, bar_color):
        ax.add_patch(patches.FancyBboxPatch((x, y), w, h, boxstyle="round,pad=0.02", ec="none", fc=color_card))
        ax.add_patch(patches.FancyBboxPatch((x, y), 0.015, h, boxstyle="round,pad=0.02", ec="none", fc=bar_color))
        ax.text(x+0.05, y+0.7*h, title, fontsize=12, color=color_primary, fontweight="bold")
        ax.text(x+0.05, y+0.35*h, value, fontsize=22, fontweight="bold")
        ax.text(x+0.05, y+0.1*h, subtext, fontsize=10, color="#858796")

    ax_k = fig.add_subplot(gs[1, 0]); ax_k.axis("off")
    draw_card(ax_k, 0.05, 0.55, 0.42, 0.35, "総返信数（KPI対象）", f"{res['total_count']:,} 件", f"翌日持ち越し除外: {res['excluded_overday']:,} 件", color_primary)
    draw_card(ax_k, 0.53, 0.55, 0.42, 0.35, "返信速度（中央値）", fmt_minutes(res['median_min']), f"平均: {fmt_minutes(res['mean_min'])}", color_success)

    fast = res["category_counts"]["5分以内"]/max(1,res["total_count"])*100
    late = res["category_counts"]["30分超"]/max(1,res["total_count"])*100
    draw_card(ax_k, 0.05, 0.1, 0.42, 0.35, "爆速返信率", f"{fast:.1f}%", "5分以内の割合", color_info)
    draw_card(ax_k, 0.53, 0.1, 0.42, 0.35, "放置率（30分超）", f"{late:.1f}%", "30分以上待たせた割合", color_danger)

    ax_p = fig.add_subplot(gs[1, 1])
    ax_p.pie(
        res['category_counts'].values,
        labels=res['category_counts'].index,
        autopct="%1.1f%%",
        startangle=90,
        counterclock=False,
        colors=[color_info, color_primary, "#858796", "#f6c23e", color_danger],
        wedgeprops={"edgecolor":"white","linewidth":3}
    )
    ax_p.add_artist(plt.Circle((0,0), 0.65, fc=color_bg))

    ax_b = fig.add_subplot(gs[2, :])
    spd = res['hourly_speed']
    b_colors = ["#e74a3b" if v > 30 else "#4e73df" for v in spd["median"]]
    ax_b.bar(spd["hour"], spd["median"], color=b_colors, alpha=0.9, label="中央値")
    ax_b.plot(spd["hour"], spd["mean"].where(spd["count"] > 0, np.nan), marker="o", color="#36b9cc", linewidth=2.5, label="平均")
    ax_b.set_title("時間帯別の返信スピード傾向（棒：中央値 / 線：平均）(赤は30分以上)", fontsize=14, fontweight="bold", loc="left")
    ax_b.set_xticks(range(24)); ax_b.grid(axis="y", linestyle="--", alpha=0.5)
    ax_b.legend()
    return fig_to_html(fig)

def create_activity_report(pid, res):
    plt.style.use("default")
    japanize_matplotlib.japanize()

    fig = plt.figure(figsize=(15, 12), dpi=120, facecolor="#F8F9FA")
    gs = fig.add_gridspec(2, 1, hspace=0.3)

    ax_a = fig.add_subplot(gs[0, 0])
    act = res['activity_stats'].reindex(range(24), fill_value=0)
    ax_a.fill_between(act.index, act["会員→パフォーマ"], alpha=0.3, label="会員の活動（受信）")
    ax_a.plot(act.index, act["パフォーマ→会員"], linewidth=2, linestyle="--", label="あなたの送信")
    ax_a.set_title("24時間のメッセージリズム"); ax_a.set_xticks(range(24)); ax_a.legend()
    ax_a.grid(axis='y', linestyle='--', alpha=0.4)

    ax_v = fig.add_subplot(gs[1, 0])
    v = res['vip_top'].sort_values("送信数", ascending=True)
    ax_v.barh(v["会員"].astype(str), v["送信数"])
    ax_v.set_title("会員からの送信数 TOP10"); ax_v.grid(axis='x', linestyle='--', alpha=0.5)
    return fig_to_html(fig)

def create_points_report(pid, points_res):
    plt.style.use("default")
    japanize_matplotlib.japanize()

    fig = plt.figure(figsize=(18, 10), dpi=130, facecolor="#F8F9FA")
    gs = fig.add_gridspec(2, 2, hspace=0.25, wspace=0.2)

    ax0 = fig.add_subplot(gs[0,0])
    msg = points_res["msg"]["by_hour"]
    ax0.bar(msg["hour"], msg["sum"], alpha=0.9)
    ax0.set_title("message_point_p：時間帯別 合計ポイント")
    ax0.set_xticks(range(24)); ax0.grid(axis="y", linestyle="--", alpha=0.4)

    ax1 = fig.add_subplot(gs[0,1])
    pay = points_res["pay"]["by_hour"]
    ax1.bar(pay["hour"], pay["sum"], alpha=0.9)
    ax1.set_title("paymail_point_p：時間帯別 合計ポイント")
    ax1.set_xticks(range(24)); ax1.grid(axis="y", linestyle="--", alpha=0.4)

    # ドーナツ円：キー別 合計ポイント（構成比）
    ax2 = fig.add_subplot(gs[1,0])
    msg_total = float(points_res["msg"]["total_points"])
    pay_total = float(points_res["pay"]["total_points"])
    grand = msg_total + pay_total

    if grand <= 0:
        ax2.axis("off")
        ax2.text(0.02, 0.55, "キー別：合計ポイント\n（データなし）", fontsize=14)
        msg_share = pay_share = 0
        pay_per_msg = np.nan
    else:
        vals = [msg_total, pay_total]
        labels = ["message_point_p", "paymail_point_p"]

        msg_share = msg_total / grand
        pay_share = pay_total / grand
        pay_per_msg = (pay_total / msg_total) if msg_total > 0 else np.nan

        ax2.pie(
            vals,
            labels=labels,
            autopct=lambda p: f"{p:.1f}%",
            startangle=90,
            counterclock=False,
            wedgeprops={"edgecolor":"white","linewidth":3}
        )
        ax2.add_artist(plt.Circle((0, 0), 0.62, fc="#F8F9FA"))
        ax2.set_title("キー別：合計ポイント（構成比）", fontsize=13)
        ax2.text(0, 0.05, f"合計\n{grand:,.0f}pt", ha="center", va="center", fontsize=13, fontweight="bold")
        ax2.text(0, -0.18, f"message {msg_total:,.0f} / pay {pay_total:,.0f}", ha="center", va="center", fontsize=10, alpha=0.85)

    ax3 = fig.add_subplot(gs[1,1]); ax3.axis("off")
    pid_label = f"ID {pid}" if pid is not None else "全体（全パフォーマ）"
    ax3.text(0.02, 0.82, f"{pid_label} ポイント概要", fontsize=16, fontweight="bold")
    ax3.text(0.02, 0.58, f"message_point_p 合計: {msg_total:,.0f} / 件数: {points_res['msg']['count']:,}", fontsize=13)
    ax3.text(0.02, 0.42, f"paymail_point_p 合計: {pay_total:,.0f} / 件数: {points_res['pay']['count']:,}", fontsize=13)
    ax3.text(0.02, 0.28, f"構成比：message {msg_share*100:.1f}% / paymail {pay_share*100:.1f}%", fontsize=13)
    ax3.text(0.02, 0.12, f"paymail / message 比: {pay_per_msg:.2f} 倍" if not np.isnan(pay_per_msg) else "paymail / message 比: -", fontsize=13)
    ax3.text(0.02, -0.02, "※ message_point_p は無料テンプレ送信が含まれません（発生分のみ）", fontsize=11, alpha=0.85)
    return fig_to_html(fig)

def create_conversion_report(conv_df):
    plt.style.use("default")
    japanize_matplotlib.japanize()

    fig = plt.figure(figsize=(18, 10), dpi=130, facecolor="#F8F9FA")
    gs = fig.add_gridspec(2, 2, hspace=0.28, wspace=0.22)

    grp = conv_df.groupby("reply_bucket").agg(
        inbound=("inbound_count","sum"),
        msg_events=("msg_events","sum"),
        pay_events=("pay_events","sum"),
        msg_points=("msg_points","sum"),
        pay_points=("pay_points","sum"),
        n=("会員ID","count")
    ).reset_index()

    order = ["5分以内","5-10分","10-15分","15-30分","30分超","返信ペアなし"]
    grp["reply_bucket"] = pd.Categorical(grp["reply_bucket"], categories=order, ordered=True)
    grp = grp.sort_values("reply_bucket")

    grp["msg_rate"] = grp.apply(lambda r: (r["msg_events"]/r["inbound"]) if r["inbound"]>0 else 0, axis=1)
    grp["pay_rate"] = grp.apply(lambda r: (r["pay_events"]/r["inbound"]) if r["inbound"]>0 else 0, axis=1)
    grp["msg_per_in"] = grp.apply(lambda r: (r["msg_points"]/r["inbound"]) if r["inbound"]>0 else 0, axis=1)
    grp["pay_per_in"] = grp.apply(lambda r: (r["pay_points"]/r["inbound"]) if r["inbound"]>0 else 0, axis=1)

    ax0 = fig.add_subplot(gs[0,0])
    ax0.bar(grp["reply_bucket"].astype(str), grp["inbound"], alpha=0.8, label="総受信（分母）")
    ax0.set_title("返信速度ビン別：総受信（棒）＋ 発生率（線）")
    ax0.grid(axis="y", linestyle="--", alpha=0.4)
    ax0.tick_params(axis='x', rotation=20)

    ax0r = ax0.twinx()
    ax0r.plot(grp["reply_bucket"].astype(str), grp["msg_rate"], marker="o", linewidth=2, label="message発生率")
    ax0r.plot(grp["reply_bucket"].astype(str), grp["pay_rate"], marker="o", linewidth=2, label="paymail発生率")
    ymax = float(grp[["msg_rate","pay_rate"]].max().max())
    ax0r.set_ylim(0, max(0.05, ymax*1.2))

    h1, l1 = ax0.get_legend_handles_labels()
    h2, l2 = ax0r.get_legend_handles_labels()
    ax0r.legend(h1+h2, l1+l2, loc="upper right")

    ax1 = fig.add_subplot(gs[0,1])
    ax1.bar(grp["reply_bucket"].astype(str), grp["msg_per_in"], alpha=0.85, label="message / 受信")
    ax1.bar(grp["reply_bucket"].astype(str), grp["pay_per_in"], alpha=0.6, label="paymail / 受信")
    ax1.set_title("返信速度ビン別：受信あたりポイント")
    ax1.grid(axis="y", linestyle="--", alpha=0.4)
    ax1.tick_params(axis='x', rotation=20)
    ax1.legend()

    ax2 = fig.add_subplot(gs[1,0])
    ax2.bar(grp["reply_bucket"].astype(str), grp["msg_points"], alpha=0.85, label="message 合計")
    ax2.bar(grp["reply_bucket"].astype(str), grp["pay_points"], alpha=0.6, label="paymail 合計")
    ax2.set_title("返信速度ビン別：合計ポイント（規模）")
    ax2.grid(axis="y", linestyle="--", alpha=0.4)
    ax2.tick_params(axis='x', rotation=20)
    ax2.legend()

    ax3 = fig.add_subplot(gs[1,1]); ax3.axis("off")
    total_in = float(conv_df["inbound_count"].sum())
    total_msg_e = float(conv_df["msg_events"].sum())
    total_pay_e = float(conv_df["pay_events"].sum())
    total_msg_p = float(conv_df["msg_points"].sum())
    total_pay_p = float(conv_df["pay_points"].sum())

    ax3.text(0.02, 0.85, "会員×日 集計（転換指標）", fontsize=16, fontweight="bold")
    ax3.text(0.02, 0.62, f"総受信: {total_in:,.0f}", fontsize=13)
    ax3.text(0.02, 0.45, f"message 発生率: {(total_msg_e/total_in*100) if total_in>0 else 0:.2f}%（件数: {total_msg_e:,.0f}）", fontsize=13)
    ax3.text(0.02, 0.28, f"paymail 発生率: {(total_pay_e/total_in*100) if total_in>0 else 0:.2f}%（件数: {total_pay_e:,.0f}）", fontsize=13)
    ax3.text(0.02, 0.12, f"合計ポイント：message {total_msg_p:,.0f} / paymail {total_pay_p:,.0f}", fontsize=13)

    return fig_to_html(fig)

# =========================
# 8. 実行本体（タブ6削除版）
# =========================
def run_full_analysis(mail_csv_path, mail_up_file, point_csv_path, point_up_file, pid_val, progress=gr.Progress()):
    t0 = time.time()
    try:
        pid = None
        if pid_val and str(pid_val).strip() != "":
            pid = int(pid_val)

        mail_path = mail_up_file.name if mail_up_file else mail_csv_path
        point_path = point_up_file.name if point_up_file else point_csv_path

        if not mail_path or not os.path.exists(mail_path):
            return status_html("error", f"メールCSVが見つかりません: {os.path.basename(str(mail_path))}"), None, None, None, None, None, None, None, None
        if not point_path or not os.path.exists(point_path):
            return status_html("error", f"ポイントCSVが見つかりません: {os.path.basename(str(point_path))}"), None, None, None, None, None, None, None, None

        progress(0.1, desc="メールCSV読み込み中...")
        df_mail = pd.read_csv(mail_path, low_memory=False)

        progress(0.2, desc="ポイントCSV読み込み中...")
        df_points = pd.read_csv(point_path, low_memory=False)

        progress(0.35, desc="返信KPI解析中...")
        reply_res, err = analyze_reply_kpi(df_mail, pid)
        if err:
            return status_html("error", err.replace("❌","").strip()), None, None, None, None, None, None, None, None

        progress(0.5, desc="ポイント解析中...")
        points_res, err = analyze_points_split(df_points, pid)
        if err:
            return status_html("error", err.replace("❌","").strip()), None, None, None, None, None, None, None, None

        progress(0.6, desc="メール分母（受信）解析中...")
        inbound_metrics, err = analyze_mail_inbound_only(df_mail, pid)
        if err:
            return status_html("error", err.replace("❌","").strip()), None, None, None, None, None, None, None, None

        progress(0.75, desc="返信×ポイント（会員×日）転換作成中...")
        conv_df = build_conversion(
            reply_res["reply_day_member"],
            inbound_metrics["inbound_day_member"],
            points_res["msg"]["day_member"],
            points_res["pay"]["day_member"]
        )

        progress(0.85, desc="レポート描画中...")
        h_reply = create_reply_report(pid, reply_res)
        h_activity = create_activity_report(pid, reply_res)
        h_points = create_points_report(pid, points_res)
        h_conv = create_conversion_report(conv_df)

        progress(0.92, desc="VIPテーブル生成中...")
        base_url = "https://own-mng.careapp.jp/flax/manage/ShowPerformerMailLog.do"

        vip_msg = reply_res["vip_top"].copy()
        vip_msg["会員ID"] = vip_msg["会員"].astype("string")
        if pid is not None:
            vip_msg["ログ確認"] = vip_msg["会員ID"].astype(str).apply(lambda m_id: f"[ログを開く]({base_url}?performerCode={pid}&memberCode={m_id})")
        else:
            vip_msg["ログ確認"] = "-"
        vip_msg_out = vip_msg[["会員ID","送信数","ログ確認"]]

        vip_msg_point = points_res["msg"]["vip"].copy()
        vip_pay_point = points_res["pay"]["vip"].copy()
        if pid is not None:
            vip_msg_point["ログ確認"] = vip_msg_point["会員ID"].astype(str).apply(lambda m_id: f"[ログを開く]({base_url}?performerCode={pid}&memberCode={m_id})")
            vip_pay_point["ログ確認"] = vip_pay_point["会員ID"].astype(str).apply(lambda m_id: f"[ログを開く]({base_url}?performerCode={pid}&memberCode={m_id})")
        else:
            vip_msg_point["ログ確認"] = "-"
            vip_pay_point["ログ確認"] = "-"

        conv_top = conv_df.copy()
        conv_top["score"] = conv_top["pay_points"] + conv_top["msg_points"]*0.2
        conv_top = conv_top.sort_values(
            ["pay_points","msg_points","inbound_count","reply_median_min"],
            ascending=[False,False,False,True]
        ).head(30)

        conv_top_out = conv_top[[
            "date","会員ID","inbound_count","reply_median_min",
            "msg_events","msg_points","pay_events","pay_points","pay_per_msg_points"
        ]].copy()

        conv_top_out = conv_top_out.rename(columns={
            "date": "日付",
            "会員ID": "会員ID",
            "inbound_count": "受信数（会員→パフォーマ）",
            "reply_median_min": "返信中央値（分）",
            "msg_events": "会話ポイント発生回数",
            "msg_points": "会話ポイント（合計）",
            "pay_events": "鑑定ポイント発生回数",
            "pay_points": "鑑定ポイント（合計）",
            "pay_per_msg_points": "鑑定/会話ポイント比",
        })

        elapsed = time.time() - t0
        pid_label = f"ID {pid}" if pid is not None else "全体（全パフォーマ）"
        sub = f"{pid_label} ｜ mail {len(df_mail):,}行 / point {len(df_points):,}行 ｜ 返信KPIペア: {reply_res['total_count']:,}件 ｜ 処理時間: {elapsed:.1f}s"

        return (
            status_html("done", "分析完了。各タブで確認してください", sub),
            h_reply,
            h_activity,
            vip_msg_out,
            h_points,
            vip_msg_point[["会員ID","合計ポイント","ログ確認"]],
            vip_pay_point[["会員ID","合計ポイント","ログ確認"]],
            h_conv,
            conv_top_out
        )
    except Exception as e:
        return status_html("error", f"エラー: {str(e)}"), None, None, None, None, None, None, None, None

# =========================
# 9. UI構築（タブ6なし）
# =========================
cands = get_csv_candidates()
choices = [(os.path.basename(p), p) for p in cands]

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 📊 統合ログ分析ダッシュボード v4.5（返信KPI + ポイント + 転換）")

    with gr.Tabs():
        with gr.TabItem("1. 設定と実行"):
            with gr.Row():
                with gr.Column():
                    gr.Markdown("### メールCSV（mailOfDay / mailOfMonth など）")
                    mail_dropdown = gr.Dropdown(
                        choices=choices,
                        value=choices[0][1] if choices else None,
                        label="メールCSV選択（共有ドライブ）"
                    )
                    mail_upload = gr.File(label="メールCSVを直接アップロード（代替）", file_types=[".csv"])

                with gr.Column():
                    gr.Markdown("### ポイントCSV（performerPointChangeOfDay など）")
                    point_dropdown = gr.Dropdown(
                        choices=choices,
                        value=choices[0][1] if choices else None,
                        label="ポイントCSV選択（共有ドライブ）"
                    )
                    point_upload = gr.File(label="ポイントCSVを直接アップロード（代替）", file_types=[".csv"])

                with gr.Column():
                    pid_input = gr.Textbox(label="パフォーマID（空欄なら全体）", placeholder="例: 12345 / 空欄で全体")
                    run_btn = gr.Button("🚀 分析を開始する", variant="primary", size="lg")
                    status_output = gr.HTML(status_html("ready", "準備OK"))

        with gr.TabItem("2. 返信速度レポート（KPI）"):
            html_reply_out = gr.HTML()

        with gr.TabItem("3. 活動傾向 ＆ 会話VIP（受信数）"):
            with gr.Row():
                with gr.Column(scale=2):
                    html_activity_out = gr.HTML()
                with gr.Column(scale=1):
                    table_vip_msg_out = gr.Dataframe(
                        headers=["会員ID", "送信数", "ログ確認"],
                        datatype=["str", "number", "markdown"],
                        interactive=False
                    )

        with gr.TabItem("4. ポイント概要 ＆ VIP（ポイント）"):
            html_points_out = gr.HTML()
            with gr.Row():
                with gr.Column():
                    gr.Markdown("### message_point_p VIP（ポイント発生分のみ）")
                    table_vip_msg_point = gr.Dataframe(
                        headers=["会員ID","合計ポイント","ログ確認"],
                        datatype=["str","number","markdown"],
                        interactive=False
                    )
                with gr.Column():
                    gr.Markdown("### paymail_point_p VIP（鑑定）")
                    table_vip_pay_point = gr.Dataframe(
                        headers=["会員ID","合計ポイント","ログ確認"],
                        datatype=["str","number","markdown"],
                        interactive=False
                    )

        with gr.TabItem("5. 返信×ポイント：転換（会員×日）"):
            html_conv_out = gr.HTML()
            table_conv_top = gr.Dataframe(
                headers=[
                    "日付","会員ID","受信数（会員→パフォーマ）","返信中央値（分）",
                    "会話ポイント発生回数","会話ポイント（合計）",
                    "鑑定ポイント発生回数","鑑定ポイント（合計）","鑑定/会話ポイント比"
                ],
                datatype=["str","str","number","number","number","number","number","number","number"],
                interactive=False
            )

    run_btn.click(
        fn=lambda: status_html("running", "分析中..."),
        outputs=status_output
    ).then(
        fn=run_full_analysis,
        inputs=[mail_dropdown, mail_upload, point_dropdown, point_upload, pid_input],
        outputs=[
            status_output,
            html_reply_out,
            html_activity_out,
            table_vip_msg_out,
            html_points_out,
            table_vip_msg_point,
            table_vip_pay_point,
            html_conv_out,
            table_conv_top
        ]
    )

# =========================
# 10. 起動
# =========================
port = 7860
while True:
    try:
        demo.launch(
            share=False,
            inline=False,
            server_name="0.0.0.0",
            server_port=port,
            prevent_thread_lock=True
        )
        break
    except:
        port += 1

from google.colab import output
output.serve_kernel_port_as_iframe(port, width="100%", height=1000)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 33.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


/tmp/ipython-input-588290387.py:639: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft()) as demo:


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>